# Forecasting with Facebook Prophet. (using daily data)
## mse= 465346.0564494159 		rmse= 682.1627785575932 	nrmse= 682.1627785575932
## (slightly better than  weekly data)

##### Reference: https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/



In [ ]:
import numpy as np 
import pandas as pd 
import fbprophet

In [ ]:
df = pd.read_csv("/content/sample_data/daily-website-visitors.csv")

df['Date'] = pd.to_datetime(df.Date, format='%m/%d/%Y')
df['Page.Loads'] = df['Page.Loads'].str.replace(',', '')
df['Count'] = df['Page.Loads'].astype(str).astype(int)
df = df[['Date', 'Count']]

# take data upto 12/31/2019
df.set_index('Date', inplace=True)
df = df[df.index <= '12/31/2019']
df.reset_index(inplace=True)

# prepare expected column names ds and y
df.columns = ['ds', 'y']
df.head()

,ds,y
0,2014-09-14,2146
1,2014-09-15,3621
2,2014-09-16,3698
3,2014-09-17,3667
4,2014-09-18,3316


In [ ]:
# train test split
Ntest = int(0.1 * len(df)) # doing 90-10 train-test split
train = df.iloc[:-Ntest]
test = df.iloc[-Ntest:]
len(train), len(test)

(1742, 193)

In [ ]:
train_idx = df.index <= train.index[-1] # ndarray of size len(df), stores True if df date (index) is <= Timestamp('2020-01-16 00:00:00') for 90-10 split
test_idx = df.index > train.index[-1] # ndarray of size len(df) for test cases. 

In [ ]:
from fbprophet import Prophet
# define the model
model = Prophet()
# fit the model
model.fit(train)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
forecast = model.predict(pd.DataFrame(test['ds']))

In [ ]:
# print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
# model.plot(forecast)
# pyplot.show()

In [ ]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(test['y'],forecast['yhat'])
rmse = math.sqrt(mse)
nrmse = rmse / (max(df['y']) - min(df['y']))
mae = mean_absolute_error(test['y'],forecast['yhat'])
print("mse=",mse,"\t\trmse=",rmse,"\tnrmse=",nrmse,"\tmae",mae)
# mse= 260351.39193944578 		rmse= 510.2464031616938 	nrmse= 0.07308026398763875 	mae 360.1932852626594

mse= 260351.39193944578 		rmse= 510.2464031616938 	nrmse= 0.07308026398763875 	mae 360.1932852626594


---
# WEEKLY DATA, SLIGHTLY MORE ERROR

---



In [ ]:
df = pd.read_csv("/content/sample_data/daily-website-visitors.csv")

df['Date'] = pd.to_datetime(df.Date, format='%m/%d/%Y')
df['Page.Loads'] = df['Page.Loads'].str.replace(',', '')
df['Count'] = df['Page.Loads'].astype(str).astype(int)
df = df[['Date', 'Day', 'Count']]
df.set_index('Date', inplace=True)

In [ ]:
# take data upto 12/31/2019
df = df[df.index <= '12/31/2019']

# drop first three data as we want len(df) % 7 = 0 for weekly data
df.drop(df.head(3).index, inplace = True)

# we now have 276 weeks: to make it pretty, let's take a whole 270 weeks and make 90-10 split.
df.drop(df.head(6*7).index, inplace = True)

# our data starts from Wednesday, so on a weekly basis date, let's take the median date of Saturdays for our index
df_saturdays = df[df['Day'] == 'Saturday']

# we will take the mean of each week to get 270 rows of weekly data
df = df.groupby(np.arange(len(df)) // 7).mean().astype(int)

# finally change the index as the corresponding Saturday dates
df.index = df_saturdays.index

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df.reset_index(inplace=True)
df.columns = ['ds', 'y']
df.head()

,ds,y
0,2014-11-01,3722
1,2014-11-08,3686
2,2014-11-15,3873
3,2014-11-22,4106
4,2014-11-29,4175


In [ ]:
Ntest = int(0.1 * len(df)) # doing 90-10 train-test split, test about 7 months
train = df.iloc[:-Ntest]
test = df.iloc[-Ntest:]
len(df), len(train), len(test)

(270, 243, 27)

In [ ]:
train_idx = df.index <= train.index[-1] # ndarray of size len(df), stores True if df date (index) is <= Timestamp('2020-01-16 00:00:00') for 90-10 split
test_idx = df.index > train.index[-1] # ndarray of size len(df) for test cases. 

In [ ]:
from fbprophet import Prophet
# define the model
model = Prophet()
# fit the model
model.fit(train)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
forecast = model.predict(pd.DataFrame(test['ds']))

In [ ]:
# print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
# model.plot(forecast)
# pyplot.show()

In [ ]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(test['y'],forecast['yhat']))
print(rmse)
# rmse on whole training set = 465.495484096981 (459 previously) , rmse on whole testing set = 722.9394245077744 (680 previously)

In [ ]:
nrmse = rmse / (max(df['y']) - min(df['y']))
print(nrmse)
# this is probably max(test['df']) - min(test['df']) 